In [1]:
import serial
import csv

arduino_port = "COM5" #serial port of Arduino
baud = 115200 #arduino uno runs at 9600 baud
fileName="analog-data.csv" #name of the CSV file generated
ser = serial.Serial(arduino_port, baud)



In [ ]:
import csv
import serial
from time import sleep

arduino_port = "COM5"  # Replace with your actual port
fileName = "sensor_data.csv"

print("Connected to Arduino port: " + arduino_port)
file = open(fileName, "a")
print("Created file")

samples = 200
line = 0
sensor_data = []

# Column names for 5 sensors + label
column_names = ["Flex1", "Flex2", "Flex3", "Flex4", "Flex5",
                "accel_x","accel_y","accel_z","gyro_x","gyro_y","gyro_z",
                "Label"]
sensor_data.append(column_names)

# Start serial connection
ser = serial.Serial(arduino_port, 115200)

# Countdown before starting
print("Starting in:")
for i in range(5, 0, -1):
    print(i)
    sleep(1)
print("Start collecting data...")

boo = True

while line < samples:
    getData = ser.readline().decode().strip()
    readings = getData.split(",")

    # Ensure correct number of readings
    if len(readings) != 11:
        continue  # skip invalid lines

    # Add label and notify user
    if line < samples / 2:
        label = "normal"
        if line == 0:
            print("Label now: normal")
    else:
        label = "กำหมัด"
        if line == samples // 2:
            print("Label now: กำหมัด")
            print("Please กำหมัด (make a fist) now!")
            sleep(2)  # Wait for user to act

    readings.append(label)
    sensor_data.append(readings)
    print(readings)

    line += 1

# Save to CSV
with open(fileName, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(sensor_data)

print("Data collection complete!")
file.close()


In [1]:
import pandas as pd

df = pd.read_csv('sensor_data.csv')
df

,Flex1,Flex2,Flex3,Flex4,Flex5,accel_x_(m/s²),accel_y_(m/s²),accel_z_(m/s²),gyro_x_(rad/s),gyro_y_(rad/s),gyro_z_(rad/s),Label
0,825,827,852,888,863,-3.4095,2.7415,-9.2467,0.4954,-0.3954,0.2926,normal
1,825,829,852,888,863,-2.2051,1.7957,-8.3393,1.7695,0.1775,0.1112,normal
2,824,830,853,889,863,-0.9266,1.5994,-8.4542,1.2619,-0.1827,0.0025,normal
3,821,830,853,889,862,0.3089,2.6146,-8.5452,-0.5742,-0.2773,-0.6593,normal
4,821,830,853,890,862,-0.7446,2.0471,-9.0672,-0.5720,1.1664,-0.5494,normal
...,...,...,...,...,...,...,...,...,...,...,...,...
195,779,687,726,843,780,-2.6816,2.3656,-8.9235,-0.1224,0.1720,0.0676,กำหมัด
196,780,687,726,843,780,-2.6912,2.2794,-8.9211,-0.1107,0.0908,-0.0273,กำหมัด
197,778,687,727,843,780,-2.5906,2.2602,-8.9403,-0.1281,0.1355,-0.0019,กำหมัด
198,779,687,727,843,780,-2.7008,2.2841,-8.9451,-0.0962,0.1003,0.0932,กำหมัด
